# XAUUSD LSTM Colab AutoLoad
مدل LSTM برای پیش‌بینی قیمت طلا (XAUUSD)

In [18]:
# این سلول برای نصب کتابخانه‌های لازم است
!pip install numpy pandas scikit-learn tensorflow matplotlib

In [19]:
# این سلول برای بارگذاری فایل CSV از GitHub است
import pandas as pd
url = 'https://raw.githubusercontent.com/behicof/new-lsdm/main/xauusd_lstm_ready_clean.csv'
df = pd.read_csv(url)
df.head()

,time,open,high,low,close,rsi,atr,ema_fast,ema_slow,ema_diff,body,upper_shadow,lower_shadow
0,2023-01-01 00:14:00,1996.550164,1996.762492,1995.565625,1996.604573,30.013488,1.394395,1999.583709,1999.910397,-0.326687,0.054409,0.157918,0.984539
1,2023-01-01 00:15:00,1998.875425,1999.709741,1998.411700,1999.457824,32.898685,1.412824,1999.568692,1999.872855,-0.304163,0.582399,0.251917,0.463725
2,2023-01-01 00:16:00,1997.974338,1998.648893,1996.986730,1998.299890,31.180558,1.399073,1999.420892,1999.747842,-0.326951,0.325552,0.349004,0.987608
3,2023-01-01 00:17:00,2000.628495,2002.103825,1999.772515,2000.482817,30.888157,1.439771,1999.542020,1999.803994,-0.261975,-0.145678,1.475330,0.710301
4,2023-01-01 00:18:00,1998.183952,1998.440806,1996.992168,1997.109391,30.847831,1.434870,1999.269670,1999.605507,-0.335837,-1.074561,0.256854,0.117223


In [17]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# انتخاب ویژگی‌ها
features = ['close', 'rsi', 'atr', 'ema_diff', 'body', 'upper_shadow', 'lower_shadow']
data = df[features].dropna()

# مقیاس‌بندی داده‌ها
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# ساخت توالی‌ها
X, y = [], []
sequence_length = 60
for i in range(sequence_length, len(scaled_data) - 10):
    X.append(scaled_data[i-sequence_length:i])
    now = data['close'].iloc[i]
    future = data['close'].iloc[i + 10]
    delta = future - now
    threshold = data['atr'].iloc[i] * 1.5
    if delta > threshold:
        y.append(2)  # سیگنال خرید
    elif delta < -threshold:
        y.append(0)  # سیگنال فروش
    else:
        y.append(1)  # نگه‌داری

X, y = np.array(X), np.array(y)

# تقسیم داده به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# ساخت مدل LSTM
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# آموزش مدل
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# ارزیابی مدل
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 263ms/step - accuracy: 0.4621 - loss: 1.0345 - val_accuracy: 0.5357 - val_loss: 1.0772
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.6427 - loss: 0.9391 - val_accuracy: 0.5357 - val_loss: 1.0283
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step - accuracy: 0.6409 - loss: 0.9107 - val_accuracy: 0.5357 - val_loss: 1.0391
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.6848 - loss: 0.8563 - val_accuracy: 0.5357 - val_loss: 1.0544
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.6646 - loss: 0.8782 - val_accuracy: 0.5357 - val_loss: 1.0246
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5100 - loss: 1.0523
Test Accuracy: 0.5357
